In [13]:
import os,sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from slr.data.ksl.keypoint_json_parser import KeypointSeq
from slr.data import datagenerator as dg


In [4]:
os.getcwd()

'C:\\Users\\user\\Documents\\GitHub\\slr_proj\\slr\\data'

In [5]:
from ksl.datapath import *

In [76]:
x,y = DataPath(100).data
KeypointSeq(x[0]).key_arr.shape, len(x)

((138, 137, 3), 8000)

In [77]:
bat = 32
for i in tqdm(range(0,len(x),32)):
    _ = [KeypointSeq(x[i_]).key_arr for i_ in range(i,i+bat)]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [03:31<00:00,  1.18it/s]


In [78]:
def make_tfrec(image,shape, label):
    return tf.train.Example(features=tf.train.Features(feature={
        'raw_data' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
        'data_shape' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[shape])),
        'label' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[label]))
    }))

In [79]:
def write_tfrecord(raw_data, labels, writer):

    for raw_d, label in zip(raw_data, labels):
        raw_d = raw_d.astype(np.float32)
        # labels = labels.astype(np.float32)
        ex = make_tfrec(raw_d.tobytes(), str(raw_d.shape).encode() ,label.encode())
        writer.write(ex.SerializeToString())


In [80]:
# write_tfrecord(data_sample,label,'test.tfrecord')

In [21]:
from contextlib import contextmanager

@contextmanager
def tf_record_writer(filename, comp = False):
    # Code to acquire resource, e.g.:
    if comp:
        writer = \
            tf.io.TFRecordWriter(
            'gzip_'+filename,
            tf.io.TFRecordOptions(compression_type="GZIP"))
    else:
        writer = tf.io.TFRecordWriter(filename)
    try:
        yield writer
    finally:
        writer.close()

In [82]:
with tf_record_writer('test.tfrec') as writer:
    for i in range(0,len(x),32):
        raw_data = np.array([
            KeypointSeq(x[i_]).key_arr 
            for i_ in range(i,i+bat)
        ])
        labels = np.array([y[i_] for i_ in range(i,i+bat)])
        write_tfrecord(raw_data, labels,writer,)

C:\Users\user\AppData\Local\Temp\ipykernel_20056\159228149.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  raw_data = np.array([


In [88]:
with tf_record_writer('test.tfrec',comp=True) as writer:
    for i in range(0,len(x),32):
        raw_data = np.array([
            KeypointSeq(x[i_]).key_arr 
            for i_ in range(i,i+bat)
        ])
        labels = np.array([y[i_] for i_ in range(i,i+bat)])
        write_tfrecord(raw_data, labels,writer,)

C:\Users\user\AppData\Local\Temp\ipykernel_20056\1718429221.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  raw_data = np.array([


## class 10 test

In [48]:
a = tf.data.TFRecordDataset('test.tfrec')

In [ ]:
for raw_record in a.take(1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

In [50]:
image_feature_description = {
    'raw_data': tf.io.FixedLenFeature([], tf.string),
    'data_shape': tf.io.FixedLenFeature([],tf.string),
    'label': tf.io.FixedLenFeature([], tf.string),
}
def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = a.map(_parse_image_function)
parsed_image_dataset

<MapDataset shapes: {data_shape: (), label: (), raw_data: ()}, types: {data_shape: tf.string, label: tf.string, raw_data: tf.string}>

In [74]:
k=0
for i in tqdm(parsed_image_dataset):
    
    raw_data = i['raw_data']
    data_shape =  eval(i['data_shape'].numpy().decode())
    # print(tf.reshape(tf.io.decode_raw(raw_data, tf.float32),data_shape).shape)
# print(k)

800it [00:00, 3043.97it/s]


## class 100 test no comp
- 원래는 3분 30초가 걸리던 load가 2초로 줄었다.

In [89]:
a = tf.data.TFRecordDataset('test.tfrec')

In [ ]:
for raw_record in a.take(1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

In [91]:
image_feature_description = {
    'raw_data': tf.io.FixedLenFeature([], tf.string),
    'data_shape': tf.io.FixedLenFeature([],tf.string),
    'label': tf.io.FixedLenFeature([], tf.string),
}
def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = a.map(_parse_image_function)
parsed_image_dataset

<MapDataset shapes: {data_shape: (), label: (), raw_data: ()}, types: {data_shape: tf.string, label: tf.string, raw_data: tf.string}>

In [92]:
k=0
for i in tqdm(parsed_image_dataset):
    
    raw_data = i['raw_data']
    data_shape =  eval(i['data_shape'].numpy().decode())
    # print(tf.reshape(tf.io.decode_raw(raw_data, tf.float32),data_shape).shape)
# print(k)

8000it [00:02, 3160.07it/s]


## class 100 test with comp

In [123]:
a = tf.data.TFRecordDataset('gzip_test.tfrec','GZIP')

In [ ]:
for raw_record in a.take(1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

In [95]:
image_feature_description = {
    'raw_data': tf.io.FixedLenFeature([], tf.string),
    'data_shape': tf.io.FixedLenFeature([],tf.string),
    'label': tf.io.FixedLenFeature([], tf.string),
}
def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = a.map(_parse_image_function)
parsed_image_dataset

<MapDataset shapes: {data_shape: (), label: (), raw_data: ()}, types: {data_shape: tf.string, label: tf.string, raw_data: tf.string}>

In [96]:
k=0
for i in tqdm(parsed_image_dataset):
    
    raw_data = i['raw_data']
    data_shape =  eval(i['data_shape'].numpy().decode())
    # print(tf.reshape(tf.io.decode_raw(raw_data, tf.float32),data_shape).shape)
# print(k)

8000it [00:02, 3165.11it/s]


In [97]:
!gzip -k test.tfrec

## class 100 save to tfrec
after seperating test,train

In [100]:
x,y = DataPath(100).data

In [103]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=66, test_size=0.3)

In [ ]:
a = [tf.ones((1,2,3)) for _ in range(10)]
b = np.array([chr(i) for i in range(10)])
for a_,b_ in zip(a,b):
    print(a_.shape, b_)

In [20]:
def make_tfrec(image,shape, label):
    return tf.train.Example(features=tf.train.Features(feature={
        'raw_data' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
        'data_shape' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[shape])),
        'label' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[label]))
    }))

def write_tfrecord(raw_data, labels, writer):

    for raw_d, label in zip(raw_data, labels):
        raw_d = raw_d.astype(np.float32)
        # labels = labels.astype(np.float32)
        ex = make_tfrec(raw_d.tobytes(), str(raw_d.shape).encode() ,label.encode())
        writer.write(ex.SerializeToString())

In [110]:
with tf_record_writer('train.tfrec',comp=True) as writer:
    for i in range(0,len(x_train),32):
        raw_data = [
            KeypointSeq(x_train[i_]).key_arr
            for i_ in range(i,i+bat)]
        labels = np.array([y_train[i_] for i_ in range(i,i+bat)])
        write_tfrecord(raw_data, labels,writer,)

In [ ]:
with tf_record_writer('test.tfrec',comp=True) as writer:
    for i in range(0,len(x_test),32):
        raw_data = [
            KeypointSeq(x_test[i_]).key_arr 
            for i_ in range(i,i+bat)]
        labels = np.array([y_test[i_] for i_ in range(i,i+bat)])
        write_tfrecord(raw_data, labels,writer,)

## tfrec with keypointgenerator

In [19]:
x,y = DataPath(100).data
bat_size=32
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=66, test_size=0.3)
train_generator = dg.KeyDataGenerator(x_train,y_train,batch_size = bat_size,scale=True,seq_len=150)
test_generator = dg.KeyDataGenerator(x_test,y_test,batch_size = bat_size,scale=True,seq_len=150)

In [30]:
for i,n in enumerate(train_generator):
    print(i)
    print(n[0].shape)
    print(n[1])
    break

0
(32, 150, 411)
[95 41 70 29 28 72 74 40 16 32 65 85  7  6 72 11 52 15 59 30 77 81 14 17
 26 97 15 83 87 22 12 82]


In [40]:
with tf_record_writer('train_with_preprocess.tfrec',comp=True) as writer:
    for n in tqdm(train_generator):
        write_tfrecord(n[0],map(str,n[1]),writer)
    # for i in range(0,len(x_train),32):
    #     raw_data = [
    #         KeypointSeq(x_train[i_]).key_arr
    #         for i_ in range(i,i+bat)]
    #     labels = np.array([y_train[i_] for i_ in range(i,i+bat)])
    #     write_tfrecord(raw_data, labels,writer,)

100%|███████████████████████████████████████████████| 175/175 [01:11<00:00,  2.46it/s]


In [33]:
with tf_record_writer('train_with_preprocess.tfrec',comp=True) as writer:
    for n in tqdm(test_generator):
        write_tfrecord(n[0],map(str,n[1]),writer)

100%|█████████████████████████████████████████████████| 75/75 [01:47<00:00,  1.44s/it]


In [41]:
test = tf.data.TFRecordDataset('gzip_train_with_preprocess.tfrec','GZIP')
image_feature_description = {
    'raw_data': tf.io.FixedLenFeature([], tf.string),
    'data_shape': tf.io.FixedLenFeature([],tf.string),
    'label': tf.io.FixedLenFeature([], tf.string),
}
def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = test.map(_parse_image_function)
parsed_image_dataset

<MapDataset shapes: {data_shape: (), label: (), raw_data: ()}, types: {data_shape: tf.string, label: tf.string, raw_data: tf.string}>

In [42]:
a=0
for i in tqdm(iter(parsed_image_dataset)):
    a+=1
print(a)

5600it [00:05, 1037.87it/s]

5600


## load from compressed tfrec

In [126]:
train = tf.data.TFRecordDataset('gzip_train.tfrec','GZIP')

In [132]:
image_feature_description = {
    'raw_data': tf.io.FixedLenFeature([], tf.string),
    'data_shape': tf.io.FixedLenFeature([],tf.string),
    'label': tf.io.FixedLenFeature([], tf.string),
}
def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = train.map(_parse_image_function)
parsed_image_dataset

<MapDataset shapes: {data_shape: (), label: (), raw_data: ()}, types: {data_shape: tf.string, label: tf.string, raw_data: tf.string}>

In [133]:
for i in tqdm(parsed_image_dataset):
    
    raw_data = i['raw_data']
    data_shape =  eval(i['data_shape'].numpy().decode())
    tf.reshape(tf.io.decode_raw(raw_data, tf.float32),data_shape).shape

5600it [00:05, 938.49it/s]


In [136]:
parsed_image_dataset.batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

<PrefetchDataset shapes: {data_shape: (None,), label: (None,), raw_data: (None,)}, types: {data_shape: tf.string, label: tf.string, raw_data: tf.string}>

In [137]:
for i in tqdm(parsed_image_dataset):
    
    raw_data = i['raw_data']
    data_shape =  eval(i['data_shape'].numpy().decode())
    tf.reshape(tf.io.decode_raw(raw_data, tf.float32),data_shape).shape

5600it [00:05, 948.05it/s]
